In [1]:
import os
import yfinance as yf
from langchain.tools import BaseTool
from typing import Any, Type
from pydantic import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import initialize_agent
from typing import Optional
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType


In [2]:
def get_stock_price(stock_ticker):   
    ticker = yf.Ticker(ticker=stock_ticker)
    current_price = ticker.history(period="1d")

    return round(current_price['Close'].iloc[-1], 2)

# Test the function
get_stock_price('AAPL')

193.6

In [3]:
def get_recommendations(recommend_stock_ticker):
     company = yf.Ticker(ticker=recommend_stock_ticker)
     return company.info['recommendationKey']


#### Test recommendations 

In [4]:
# Test on 12/2023
print(get_recommendations('AAPL'))
print(get_recommendations('SHOP'))

print(get_recommendations('MO'))
print(get_recommendations('CGC'))
#underperform, buy, hold

buy
buy
hold
underperform


In [5]:
class StockPriceCheckInput(BaseModel):
    """Input to check Stock price."""

    stock_ticker: str = Field(description="Ticker symbol for stock")

In [6]:
class StockPriceTool(BaseTool):
    name = "get_stock_price"
    description = """
    Useful to find the stock price of a company and also suggest to buy number of stocks if asked. You should input the stock ticker which is used by yahoo finance API
    """
    
    def _run(self, stock_ticker: str):
        stock_price = get_stock_price(stock_ticker)
        return stock_price

    def _arun(self, stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput   

In [7]:
class RecommendationStockInput(BaseModel):
    """Input to the stock recommendations"""

    recommend_stock_ticker: str = Field(description="Ticker symbol for stock")

In [8]:
class RecommendationStockTool(BaseTool):
    name = "get_recommendations"
    description = "Useful to find recommendations to buy, hold if you have stocks or not to buy, You should input the stock ticker which is used by yahoo finance API"
    
    def _run(self, recommend_stock_ticker: str):
        stock_recommended_to_buy = get_recommendations(recommend_stock_ticker)
        return stock_recommended_to_buy

    def _arun(self, recommend_stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Optional[Type[BaseModel]] = RecommendationStockInput   

### Tools as openAI functions

In [9]:
stock_tools = [StockPriceTool(), RecommendationStockTool()]
functions = [format_tool_to_openai_function(fun) for fun in stock_tools]

In [10]:
functions

[{'name': 'get_stock_price',
  'description': '\n    Useful to find the stock price of a company and also suggest to buy number of stocks if asked. You should input the stock ticker which is used by yahoo finance API\n    ',
  'parameters': {'description': 'Input to check Stock price.',
   'properties': {'stock_ticker': {'description': 'Ticker symbol for stock',
     'title': 'Stock Ticker',
     'type': 'string'}},
   'required': ['stock_ticker'],
   'title': 'StockPriceCheckInput',
   'type': 'object'}},
 {'name': 'get_recommendations',
  'description': 'Useful to find recommendations to buy, hold if you have stocks or not to buy, You should input the stock ticker which is used by yahoo finance API',
  'parameters': {'description': 'Input to the stock recommendations',
   'properties': {'recommend_stock_ticker': {'description': 'Ticker symbol for stock',
     'title': 'Recommend Stock Ticker',
     'type': 'string'}},
   'required': ['recommend_stock_ticker'],
   'title': 'Recommenda

### Langchain Agent

In [31]:
llm = ChatOpenAI( api_key=os.environ['API_Key'], 
                   temperature=0 )

open_ai_agent = initialize_agent(tools=stock_tools, 
                                 model = 'gpt-3.5-turbo',
                                 llm=llm, 
                                 agent=AgentType.OPENAI_MULTI_FUNCTIONS, 
                                 verbose=True)

### Get stock price.
LLM's are smart to convert company name to stock ticker

In [12]:
agent_results = open_ai_agent.run("what is the latest price of Amazon stock")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'AMZN'}`


153.42The latest price of Amazon stock is $153.42.

> Finished chain.


'The latest price of Amazon stock is $153.42.'

### Get recommendation from LLM to buy a stock.
LLM's will be using yahoo finance api's recommendation

In [14]:
agent_results = open_ai_agent.run("what is the latest price of Canopy Growth stock, is it recommended to buy ?")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'CGC'}`


5.14
Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'CGC'}`


underperformThe latest price of Canopy Growth stock (CGC) is $5.14. However, the recommendation for this stock is "underperform", which means it is not recommended to buy at the moment.

> Finished chain.


'The latest price of Canopy Growth stock (CGC) is $5.14. However, the recommendation for this stock is "underperform", which means it is not recommended to buy at the moment.'

### Number of stocks you can buy
Check to see if LLM can suggest the number of stocks to buy

In [32]:
agent_results = open_ai_agent.run("I am planning to invest $5000 in apple stock, how may stocks can I buy and is it recommended to invest in apple stocks")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'AAPL'}`


193.6
Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'AAPL'}`


buyThe current price of Apple stock (AAPL) is $193.6. With an investment of $5000, you can buy approximately 25.84 shares of Apple stock.

As for the recommendation, the tool suggests buying Apple stock.

> Finished chain.


'The current price of Apple stock (AAPL) is $193.6. With an investment of $5000, you can buy approximately 25.84 shares of Apple stock.\n\nAs for the recommendation, the tool suggests buying Apple stock.'

In [35]:
print(agent_results)

The current price of Canopy Growth stock (CGC) is $5.14. With an investment of $1000, you can buy approximately 194 stocks (1000 / 5.14 = 194.17).

As for the recommendation, the stock is currently rated as "underperform". It is important to consider this information before making any investment decisions.


### Invest money and check how may stock to buy

In [36]:
agent_results = open_ai_agent.run("I am planning to invest $5000 in apple stock, how may stocks can I buy ? ")



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'AAPL'}`


193.6The current price of Apple stock is $193.6. To calculate how many stocks you can buy with $5000, divide the total amount by the stock price:

$5000 / $193.6 = 25.84

Since you cannot buy a fraction of a stock, you can buy a maximum of 25 shares of Apple stock with $5000.

> Finished chain.


In [38]:
print(agent_results)

The current price of Apple stock is $193.6. To calculate how many stocks you can buy with $5000, divide the total amount by the stock price:

$5000 / $193.6 = 25.84

Since you cannot buy a fraction of a stock, you can buy a maximum of 25 shares of Apple stock with $5000.


### Try to buy underperformed stock

In [40]:
agent_results = open_ai_agent.run("I am planning to invest $1000 in Canopy Growth stock, how may stocks can I buy and is it recommended to invest in these stocks")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'CGC'}`


5.14
Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'CGC'}`


underperformThe current price of Canopy Growth stock (CGC) is $5.14. With an investment of $1000, you can buy approximately 194 stocks (1000 / 5.14 = 194.17).

As for the recommendation, the stock is currently rated as "underperform". It is important to consider this information before making any investment decisions.

> Finished chain.


'The current price of Canopy Growth stock (CGC) is $5.14. With an investment of $1000, you can buy approximately 194 stocks (1000 / 5.14 = 194.17).\n\nAs for the recommendation, the stock is currently rated as "underperform". It is important to consider this information before making any investment decisions.'

In [41]:
print(agent_results)

The current price of Canopy Growth stock (CGC) is $5.14. With an investment of $1000, you can buy approximately 194 stocks (1000 / 5.14 = 194.17).

As for the recommendation, the stock is currently rated as "underperform". It is important to consider this information before making any investment decisions.
